In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import urllib
from shapely.geometry import Point
import geopy.distance
from geopy import distance
from geopy import Point
import matplotlib.pyplot as plt 
from scipy import stats
import seaborn as sns
import geocoder
import json
import urllib
import urllib.request
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer

The Point of this notebook is to look at which attribute is the best predictor for the type of bikes

In [3]:
bikes=pd.read_csv('bikes_full_SF_Feb_ml.csv', error_bad_lines=False)
elevation=pd.read_csv('elevation_sf.csv', error_bad_lines=False)

In [8]:
elevation.set_index('station_id',inplace=True)
elevation

,SF-M26,SF-M25,SF-M26-2,SF-L27,SF-K26,SF-J27,SF-M30,SF-L27-2,SF-J29-3,SF-J29,...,SF-Z7,SF-Z6,SF-W8,SF-BB17,SF-AA15,SF-Z16,SF-Y14,SF-X14,SF-X19,SF-Y25
station_id,,,,,,,,,,,,,,,,,,,,,
SF-M26,0.0,3.0,-8.0,-8.0,-8.0,-8.0,-8.0,-8.0,-8.0,-8.0,...,28.0,19.0,50.0,52.0,59.0,44.0,87.0,90.0,25.0,15.0
SF-M25,-3.0,0.0,-11.0,-11.0,-11.0,-11.0,-11.0,-11.0,-11.0,-11.0,...,25.0,16.0,47.0,49.0,56.0,41.0,84.0,87.0,22.0,12.0
SF-M26-2,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,27.0,58.0,60.0,67.0,52.0,95.0,98.0,33.0,23.0
SF-L27,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,27.0,58.0,60.0,67.0,52.0,95.0,98.0,33.0,23.0
SF-K26,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,27.0,58.0,60.0,67.0,52.0,95.0,98.0,33.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF-Z16,-44.0,-41.0,-52.0,-52.0,-52.0,-52.0,-52.0,-52.0,-52.0,-52.0,...,-16.0,-25.0,6.0,8.0,15.0,0.0,43.0,46.0,-19.0,-29.0
SF-Y14,-87.0,-84.0,-95.0,-95.0,-95.0,-95.0,-95.0,-95.0,-95.0,-95.0,...,-59.0,-68.0,-37.0,-35.0,-28.0,-43.0,0.0,3.0,-62.0,-72.0
SF-X14,-90.0,-87.0,-98.0,-98.0,-98.0,-98.0,-98.0,-98.0,-98.0,-98.0,...,-62.0,-71.0,-40.0,-38.0,-31.0,-46.0,-3.0,0.0,-65.0,-75.0


In [9]:
bikes['started_at']=pd.to_datetime(bikes['started_at'],errors='coerce')
bikes['ended_at']=pd.to_datetime(bikes['ended_at'],errors='coerce')
bikes['start_hour']=bikes['started_at'].dt.hour
bikes['end_hour']=bikes['ended_at'].dt.hour
bikes

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,displacement,duration,velocity,start_hour,end_hour,type_num
0,072879C2ED883CF1,classic_bike,2021-02-12 19:08:22,2021-02-12 19:15:02,Townsend St at 5th St,SF-K28,Mississippi St at 17th St,SF-N29,37.775214,-122.397477,37.764793,-122.394796,member,1.180490e+06,400.0,2951.224312,19,19,1
1,25D1A5D72C59C493,classic_bike,2021-02-28 10:34:39,2021-02-28 11:00:01,19th St at Florida St,SF-O25-1,2nd St at Folsom St,SF-H29,37.760437,-122.410778,37.785318,-122.396434,member,3.037043e+06,1522.0,1995.429149,10,11,1
2,5D7DCF49E7C54D43,electric_bike,2021-02-14 13:44:57,2021-02-14 13:51:55,19th St at Florida St,SF-O25-1,Bryant St at 15th St,SF-M25,37.760437,-122.410778,37.767100,-122.410662,member,7.396375e+05,418.0,1769.467733,13,13,0
3,EF8B274C5CD4559E,electric_bike,2021-02-22 18:24:57,2021-02-22 18:50:10,Utah St at 24th St,SF-Q26,24th St at Bartlett St,SF-Q23-3,37.753243,-122.405496,37.752131,-122.419883,member,1.273815e+06,1513.0,841.913402,18,18,0
4,A896A2BE15365157,classic_bike,2021-02-22 13:16:00,2021-02-22 13:21:37,19th St at Florida St,SF-O25-1,16th St Mission BART Station 2,SF-N22-1B,37.760437,-122.410778,37.764854,-122.420129,member,9.587779e+05,337.0,2845.038145,13,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60274,6CC0C441607E4BAB,classic_bike,2021-02-18 06:02:39,2021-02-18 06:22:48,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1209.0,2891.959774,6,6,1
60275,ECC333B00108B2EB,classic_bike,2021-02-27 06:25:11,2021-02-27 06:40:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,923.0,3788.059986,6,6,1
60276,63971FE6AFC02507,classic_bike,2021-02-25 06:25:20,2021-02-25 06:41:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,974.0,3589.711876,6,6,1
60277,D343E2081AB58E27,classic_bike,2021-02-28 06:13:37,2021-02-28 06:31:12,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1055.0,3314.103665,6,6,1


In [11]:
elevation.loc['SF-M26']['SF-M25']

3.0

In [23]:
bikes['ele_diff']=0
for index,row in bikes.iterrows():
    try:
        bikes.loc[index,'ele_diff']=elevation.loc[row['start_station_id']][row['end_station_id']]
        print(bikes.loc[index,'ele_diff'])
    except KeyError:
        pass

bikes

7.0
8.0
6.0
8.0
2.0
-79.0
-55.0
23.0
16.0
7.0
-10.0
0.0
40.0
1.0
44.0
44.0
44.0
-4.0
7.0
7.0
7.0
7.0
-12.0
3.0
-38.0
-51.0
-51.0
3.0
-14.0
-14.0
-11.0
-51.0
-51.0
-47.0
-47.0
-76.0
-3.0
-2.0
3.0
-29.0
-29.0
-54.0
-54.0
-55.0
-55.0
16.0
16.0
17.0
-10.0
-18.0
-42.0
-20.0
-20.0
-3.0
-4.0
-4.0
-4.0
-4.0
-4.0
0.0
-51.0
13.0
-61.0
-61.0
-26.0
-26.0
-26.0
-26.0
-26.0
-9.0
-51.0
13.0
-10.0
15.0
-33.0
-33.0
-33.0
-8.0
10.0
59.0
49.0
54.0
31.0
31.0
42.0
10.0
-37.0
-12.0
-12.0
-12.0
-25.0
-25.0
-25.0
4.0
-2.0
1.0
1.0
1.0
0.0
-3.0
-28.0
-17.0
-45.0
22.0
-12.0
-4.0
-4.0
-12.0
-79.0
-12.0
-4.0
-29.0
-29.0
-29.0
-29.0
27.0
27.0
27.0
-15.0
-15.0
-15.0
27.0
27.0
27.0
12.0
6.0
6.0
6.0
6.0
6.0
-2.0
14.0
6.0
6.0
-42.0
11.0
-8.0
7.0
7.0
7.0
-69.0
-14.0
-9.0
-33.0
1.0
4.0
-7.0
8.0
8.0
-83.0
-58.0
-10.0
-83.0
-83.0
11.0
9.0
9.0
9.0
9.0
16.0
9.0
22.0
22.0
-25.0
6.0
-11.0
13.0
-21.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
19.0
13.0
13.0
1.0
1.0
1.0
-10.0
-20.0
-20.0
-20.0
-20.0
-20.0
-22.0
-11.0
1.0
33.0
6.0
40.0
40.0
40.

-8.0
7.0
-29.0
-29.0
19.0
10.0
29.0
-14.0
10.0
-21.0
-21.0
-21.0
2.0
4.0
-31.0
-31.0
-31.0
-31.0
14.0
14.0
11.0
42.0
42.0
7.0
-32.0
-26.0
-39.0
-39.0
-39.0
-15.0
-15.0
-15.0
-15.0
-15.0
-15.0
-15.0
-15.0
-15.0
-15.0
9.0
-4.0
9.0
9.0
9.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
9.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
7.0
0.0
-82.0
11.0
-18.0
-18.0
-5.0
-18.0
11.0
-18.0
-18.0
-2.0
-2.0
-5.0
-8.0
10.0
10.0
10.0
10.0
0.0
0.0
1.0
-48.0
-31.0
-36.0
-6.0
0.0
3.0
3.0
7.0
-19.0
11.0
7.0
7.0
7.0
7.0
-23.0
-10.0
-10.0
-14.0
-10.0
-10.0
-14.0
-14.0
-14.0
-12.0
-26.0
-30.0
-13.0
4.0
-35.0
-2.0
-4.0
36.0
1.0
41.0
-12.0
45.0
10.0
-3.0
-3.0
-23.0
12.0
12.0
55.0
-2.0
70.0
-6.0
-3.0
14.0
14.0
-2.0
33.0
-6.0
-31.0
-3.0
-6.0
-6.0
46.0
-35.0
69.0
16.0
17.0
17.0
16.0
11.0
11.0
16.0
16.0
11.0
16.0
8.0
8.0
8.0
13.0
-17.0
12.0
12.0
-3.0
-12.0
7.0
2.0
-18.0
-69.0
-46.0
-69.0
-46.0
13.0
13.0
13.0
-8.0
6.0
6.0
6.0
-2.0
-2.0
14.0
9.0
-5.0
-7.0
12.0
12.0
10.0
12.0
-5.0
-7.0
-7.0
-7.0
-7.0
-5.

4.0
-18.0
27.0
35.0
1.0
4.0
1.0
1.0
1.0
27.0
9.0
27.0
6.0
14.0
-36.0
8.0
-30.0
16.0
16.0
-13.0
16.0
16.0
-29.0
0.0
42.0
-29.0
0.0
-32.0
-32.0
-32.0
0.0
-29.0
26.0
42.0
-31.0
-31.0
-84.0
53.0
-39.0
-39.0
-81.0
-5.0
-68.0
-8.0
24.0
-67.0
-15.0
3.0
3.0
3.0
0.0
3.0
0.0
0.0
0.0
0.0
3.0
3.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-2.0
-2.0
3.0
-2.0
-2.0
-2.0
-2.0
-2.0
-39.0
3.0
-5.0
3.0
3.0
-2.0
51.0
-39.0
11.0
0.0
3.0
3.0
3.0
3.0
-5.0
-5.0
-3.0
-43.0
45.0
-3.0
-3.0
-39.0
14.0
27.0
-10.0
1.0
1.0
-16.0
-19.0
-19.0
-19.0
-19.0
-19.0
-19.0
-19.0
-19.0
-35.0
-12.0
-31.0
-39.0
-39.0
-39.0
-39.0
-39.0
-31.0
1.0
-3.0
-3.0
-65.0
-15.0
2.0
-3.0
26.0
-1.0
-17.0
22.0
1.0
1.0
4.0
-2.0
4.0
1.0
1.0
1.0
1.0
36.0
0.0
-6.0
0.0
-27.0
-69.0
-35.0
-35.0
-4.0
-35.0
-35.0
-46.0
-15.0
-15.0
-15.0
-15.0
-4.0
-15.0
-3.0
0.0
-5.0
8.0
-30.0
16.0
-16.0
-16.0
-16.0
34.0
-13.0
-82.0
34.0
34.0
34.0
-3.0
-3.0
-16.0
-3.0
-49.0
-35.0
-4.0
-77.0
-27.0
-35.0
3.0
-20.0
45.0
68.0
45.0
45.0
1.0
42.0
42.0
42.0
81.0
-82.0
21.0
21.0
21.

-1.0
-6.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-10.0
-3.0
-3.0
-3.0
-25.0
-25.0
-25.0
-25.0
-25.0
-37.0
-22.0
-22.0
-44.0
23.0
-54.0
0.0
-9.0
-9.0
-9.0
-10.0
15.0
1.0
-45.0
-9.0
-40.0
1.0
0.0
-57.0
-58.0
-83.0
-31.0
-57.0
-57.0
-57.0
-57.0
-12.0
-1.0
21.0
11.0
-39.0
-13.0
-39.0
-39.0
-39.0
-39.0
-39.0
-39.0
-39.0
-39.0
-39.0
-39.0
-39.0
-39.0
-39.0
-81.0
23.0
-59.0
-9.0
-4.0
11.0
-86.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
28.0
-4.0
-4.0
28.0
28.0
51.0
28.0
-4.0
-4.0
-4.0
-4.0
-5.0
-11.0
4.0
4.0
4.0
-58.0
-32.0
0.0
0.0
-18.0
-19.0
-19.0
-19.0
-19.0
-19.0
-19.0
-13.0
4.0
1.0
4.0
-14.0
-4.0
1.0
-14.0
-5.0
1.0
-5.0
0.0
-4.0
-4.0
0.0
-14.0
-14.0
-24.0
-80.0
-4.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
-29.0
-29.0
-29.0
-29.0
-29.0
-29.0
-29.0
-29.0
2.0
-12.0
53.0
-3.0
16.0
0.0
-3.0
-23.0
13.0
2.0
2.0
21.0
-1.0
3.0
3.0
3.0
3.0
58.0
2.0
55.0
-18.0
2.0
-11.0
45.0
3.0
3.0
3.0
2.0
-26.0
-34.0
58.0
27.0
0.0
-13.0
0.0
-40.0
19.0
-30.0
3.0
-50.0
-50.0
9.0
2.0
2.0
-5.0
-5.0
-5.0
-

-5.0
-2.0
-3.0
0.0
0.0
-1.0
-25.0
-25.0
-25.0
-25.0
-25.0
-25.0
-25.0
-14.0
-11.0
-15.0
20.0
-39.0
0.0
0.0
0.0
0.0
0.0
-2.0
0.0
-2.0
0.0
-39.0
-39.0
0.0
-30.0
-2.0
-10.0
-6.0
-12.0
-10.0
-1.0
-1.0
0.0
-3.0
-3.0
-8.0
-7.0
17.0
-2.0
-3.0
-3.0
-8.0
27.0
-8.0
27.0
1.0
-9.0
-47.0
-57.0
-47.0
-82.0
-82.0
-82.0
-47.0
-14.0
-34.0
-34.0
2.0
-33.0
-33.0
-3.0
-32.0
-32.0
-32.0
1.0
1.0
0.0
32.0
32.0
32.0
0.0
0.0
0.0
32.0
-52.0
0.0
-9.0
-9.0
27.0
-4.0
0.0
0.0
-9.0
-10.0
-9.0
-9.0
-10.0
-9.0
-9.0
-9.0
-9.0
-1.0
-10.0
-10.0
-10.0
36.0
36.0
36.0
36.0
-1.0
1.0
1.0
1.0
1.0
1.0
-1.0
1.0
-36.0
0.0
-1.0
-3.0
-2.0
-2.0
-57.0
15.0
0.0
3.0
1.0
1.0
36.0
36.0
36.0
36.0
-9.0
-9.0
-9.0
-2.0
-4.0
-32.0
-32.0
-25.0
1.0
18.0
35.0
35.0
35.0
35.0
35.0
35.0
61.0
61.0
61.0
61.0
94.0
-23.0
50.0
7.0
7.0
4.0
-49.0
-49.0
-49.0
-49.0
35.0
35.0
35.0
35.0
53.0
53.0
53.0
53.0
35.0
35.0
35.0
75.0
-48.0
16.0
84.0
0.0
0.0
45.0
45.0
-42.0
-32.0
-32.0
-3.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-31.0
-31.0
-31.0
1.0
1.0
1.0
-25.0
-25.0

...

1.0
1.0
1.0
1.0
1.0
-2.0
-2.0
-2.0
-2.0
-2.0
7.0
7.0
16.0
-17.0
7.0
-17.0
-2.0
-2.0
-2.0
7.0
7.0
7.0
-2.0
6.0
-16.0
-2.0
7.0
6.0
-23.0
-2.0
-16.0
6.0
-2.0
19.0
-2.0
7.0
7.0
-22.0
-16.0
31.0
-31.0
-16.0
-17.0
-17.0
-17.0
21.0
5.0
5.0
5.0
32.0
21.0
9.0
9.0
9.0
9.0
8.0
46.0
1.0
1.0
-7.0
8.0
9.0
9.0
10.0
10.0
17.0
17.0
13.0
33.0
36.0
-22.0
43.0
43.0
43.0
43.0
4.0
4.0
4.0
33.0
33.0
33.0
33.0
4.0
38.0
42.0
4.0
-17.0
3.0
17.0
-22.0
43.0
42.0
42.0
33.0
41.0
48.0
9.0
48.0
9.0
4.0
2.0
4.0
2.0
4.0
4.0
4.0
4.0
4.0
4.0
2.0
2.0
62.0
62.0
2.0
41.0
2.0
48.0
67.0
67.0
67.0
67.0
7.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
1.0
1.0
4.0
4.0
12.0
12.0
-3.0
-6.0
-6.0
12.0
-6.0
-4.0
-4.0
34.0
-4.0
-6.0
-1.0
-1.0
4.0
16.0
-2.0
4.0
16.0
-5.0
-5.0
27.0
16.0
-5.0
27.0
44.0
44.0
-5.0
17.0
17.0
-2.0
-2.0
-2.0
6.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-31.0
-16.0
-16.0
-23.0
-17.0
7.0
-25.0
-3.0
66.0
66.0
14.0
14.0
14.0
20.0
-65.0
13.0
-24.0
14.0
14.0
8.0
-12.0
13.

-4.0
-13.0
-13.0
-64.0
2.0
2.0
49.0
8.0
-64.0
11.0
-30.0
-20.0
-17.0
11.0
-64.0
-76.0
-23.0
-77.0
-64.0
-9.0
-9.0
-9.0
-9.0
-9.0
12.0
-9.0
-14.0
53.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
4.0
4.0
4.0
4.0
4.0
-12.0
-12.0
-12.0
-12.0
-12.0
-3.0
11.0
10.0
51.0
2.0
-12.0
-12.0
4.0
-3.0
-3.0
2.0
5.0
4.0
5.0
5.0
5.0
5.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
2.0
2.0
2.0
-12.0
-12.0
-12.0
-4.0
5.0
4.0
-12.0
-11.0
-11.0
-11.0
-11.0
-11.0
-11.0
-11.0
-6.0
2.0
-5.0
3.0
-46.0
-46.0
-3.0
-11.0
-11.0
51.0
-2.0
-2.0
-44.0
-44.0
-44.0
-44.0
-44.0
-35.0
-35.0
-35.0
-44.0
-35.0
-35.0
-35.0
-20.0
13.0
16.0
-46.0
1.0
-8.0
0.0
0.0
0.0
0.0
0.0
0.0
16.0
-8.0
-8.0
-8.0
-8.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
-8.0
-8.0
-8.0
-8.0
1.0
-8.0
-8.0
-46.0
-46.0
-46.0
-46.0
16.0
-37.0
-29.0
24.0
23.0
24.0
-29.0
-29.0
-29.0
-29.0
24.0
-17.0
-17.0
-30.0
-17.0
-3.0
-12.0
-4.0
-12.0
-12.0
-12.0
-12.0
-12.0
-12.0
-12.0
-3.0
-12.0
-12.0
-12.0
45.0
45.0
-9.0
-41.0
-41.0

18.0
18.0
-11.0
9.0
18.0
10.0
-3.0
-11.0
18.0
18.0
18.0
18.0
34.0
34.0
34.0
34.0
26.0
34.0
34.0
34.0
-28.0
34.0
-28.0
34.0
34.0
34.0
34.0
34.0
26.0
-24.0
-24.0
7.0
-3.0
15.0
14.0
76.0
7.0
0.0
-58.0
-58.0
-58.0
30.0
1.0
1.0
-2.0
1.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-1.0
12.0
-1.0
-10.0
-10.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-22.0
-10.0
-59.0
-59.0
-3.0
0.0
7.0
4.0
7.0
-55.0
-55.0
-55.0
-55.0
-55.0
-55.0
-14.0
-3.0
-3.0
-3.0
-3.0
-3.0
-3.0
-3.0
-3.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
-1.0
2.0
-7.0
2.0
38.0
1.0
1.0
1.0
1.0
1.0
7.0
7.0
2.0
2.0
16.0
16.0
-50.0
11.0
11.0
-50.0
-50.0
11.0
11.0
8.0
11.0
-50.0
8.0
11.0
-13.0
0.0
8.0
2.0
35.0
-12.0
-23.0
-23.0
-23.0
-23.0
-23.0
-1.0
-1.0
2.0
-10.0
-4.0
-7.0
7.0
2.0
-7.0
4.0
-1.0
-1.0
-1.0
-7.0
-1.0
-1.0
-23.0
2.0
2.0
2.0
2.0
-1.0
-3.0
-3.0
-3.0
-10.0
23.0
-6.0
2.0
-7.0
-7.0
-7.0
2.0
2.0
2.0
-10.0
-14.0
-14.0
-14.0
-14.0
-6.0
-56.0
-56.0
-6.0
33.0
44.0
-18.0
4.0
4.0
-1.0
-18.0
-12.0
0.0
0.0
-8.0
-8.0
-1.0
34.0
-1.0
-1.0
-1.0
-1.0
-1

10.0
10.0
10.0
-9.0
2.0
12.0
23.0
-24.0
-24.0
7.0
2.0
-2.0
-13.0
34.0
34.0
34.0
34.0
34.0
-7.0
2.0
2.0
2.0
2.0
2.0
-7.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
-2.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
-11.0
1.0
34.0
-10.0
43.0
31.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
-4.0
-4.0
25.0
34.0
20.0
35.0
31.0
9.0
9.0
9.0
20.0
34.0
13.0
-2.0
26.0
34.0
25.0
25.0
34.0
34.0
-28.0
-28.0
13.0
-24.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
31.0
31.0
-1.0
-1.0
-1.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
-28.0
25.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
-13.0
-13.0
8.0
-58.0
-50.0
8.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
-50.0
8.0
8.0
8.0
8.0
8.0
-50.0
-50.0
-50.0
-50.0
-50.0
34.0
-50.0
-2.0
1.0
1.0
-1.0
-1.0
5.0
-2.0
13.0
-9.0
-9.0
13.0
4.0
-54.0
-54.0
-54.0
-54.0

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,displacement,duration,velocity,start_hour,end_hour,type_num,ele_diff
0,072879C2ED883CF1,classic_bike,2021-02-12 19:08:22,2021-02-12 19:15:02,Townsend St at 5th St,SF-K28,Mississippi St at 17th St,SF-N29,37.775214,-122.397477,37.764793,-122.394796,member,1.180490e+06,400.0,2951.224312,19,19,1,7.0
1,25D1A5D72C59C493,classic_bike,2021-02-28 10:34:39,2021-02-28 11:00:01,19th St at Florida St,SF-O25-1,2nd St at Folsom St,SF-H29,37.760437,-122.410778,37.785318,-122.396434,member,3.037043e+06,1522.0,1995.429149,10,11,1,8.0
2,5D7DCF49E7C54D43,electric_bike,2021-02-14 13:44:57,2021-02-14 13:51:55,19th St at Florida St,SF-O25-1,Bryant St at 15th St,SF-M25,37.760437,-122.410778,37.767100,-122.410662,member,7.396375e+05,418.0,1769.467733,13,13,0,6.0
3,EF8B274C5CD4559E,electric_bike,2021-02-22 18:24:57,2021-02-22 18:50:10,Utah St at 24th St,SF-Q26,24th St at Bartlett St,SF-Q23-3,37.753243,-122.405496,37.752131,-122.419883,member,1.273815e+06,1513.0,841.913402,18,18,0,8.0
4,A896A2BE15365157,classic_bike,2021-02-22 13:16:00,2021-02-22 13:21:37,19th St at Florida St,SF-O25-1,16th St Mission BART Station 2,SF-N22-1B,37.760437,-122.410778,37.764854,-122.420129,member,9.587779e+05,337.0,2845.038145,13,13,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60274,6CC0C441607E4BAB,classic_bike,2021-02-18 06:02:39,2021-02-18 06:22:48,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1209.0,2891.959774,6,6,1,-71.0
60275,ECC333B00108B2EB,classic_bike,2021-02-27 06:25:11,2021-02-27 06:40:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,923.0,3788.059986,6,6,1,-71.0
60276,63971FE6AFC02507,classic_bike,2021-02-25 06:25:20,2021-02-25 06:41:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,974.0,3589.711876,6,6,1,-71.0
60277,D343E2081AB58E27,classic_bike,2021-02-28 06:13:37,2021-02-28 06:31:12,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1055.0,3314.103665,6,6,1,-71.0


In [24]:
bikes['type_num']=0
bikes['type_num']=[1 if x=='classic_bike' else 0 for x in bikes['rideable_type']]
bikes

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,displacement,duration,velocity,start_hour,end_hour,type_num,ele_diff
0,072879C2ED883CF1,classic_bike,2021-02-12 19:08:22,2021-02-12 19:15:02,Townsend St at 5th St,SF-K28,Mississippi St at 17th St,SF-N29,37.775214,-122.397477,37.764793,-122.394796,member,1.180490e+06,400.0,2951.224312,19,19,1,7.0
1,25D1A5D72C59C493,classic_bike,2021-02-28 10:34:39,2021-02-28 11:00:01,19th St at Florida St,SF-O25-1,2nd St at Folsom St,SF-H29,37.760437,-122.410778,37.785318,-122.396434,member,3.037043e+06,1522.0,1995.429149,10,11,1,8.0
2,5D7DCF49E7C54D43,electric_bike,2021-02-14 13:44:57,2021-02-14 13:51:55,19th St at Florida St,SF-O25-1,Bryant St at 15th St,SF-M25,37.760437,-122.410778,37.767100,-122.410662,member,7.396375e+05,418.0,1769.467733,13,13,0,6.0
3,EF8B274C5CD4559E,electric_bike,2021-02-22 18:24:57,2021-02-22 18:50:10,Utah St at 24th St,SF-Q26,24th St at Bartlett St,SF-Q23-3,37.753243,-122.405496,37.752131,-122.419883,member,1.273815e+06,1513.0,841.913402,18,18,0,8.0
4,A896A2BE15365157,classic_bike,2021-02-22 13:16:00,2021-02-22 13:21:37,19th St at Florida St,SF-O25-1,16th St Mission BART Station 2,SF-N22-1B,37.760437,-122.410778,37.764854,-122.420129,member,9.587779e+05,337.0,2845.038145,13,13,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60274,6CC0C441607E4BAB,classic_bike,2021-02-18 06:02:39,2021-02-18 06:22:48,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1209.0,2891.959774,6,6,1,-71.0
60275,ECC333B00108B2EB,classic_bike,2021-02-27 06:25:11,2021-02-27 06:40:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,923.0,3788.059986,6,6,1,-71.0
60276,63971FE6AFC02507,classic_bike,2021-02-25 06:25:20,2021-02-25 06:41:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,974.0,3589.711876,6,6,1,-71.0
60277,D343E2081AB58E27,classic_bike,2021-02-28 06:13:37,2021-02-28 06:31:12,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1055.0,3314.103665,6,6,1,-71.0


Create a Column for quantifiable predction, if bike is classic then 1 else 0

In [25]:
X=pd.DataFrame(bikes['ele_diff'])
Y=pd.DataFrame(bikes['type_num'])

Checking accuracy for displacement.

In [26]:
X

,ele_diff
0,7.0
1,8.0
2,6.0
3,8.0
4,2.0
...,...
60274,-71.0
60275,-71.0
60276,-71.0
60277,-71.0


In [27]:
Y

,type_num
0,1
1,1
2,0
3,0
4,1
...,...
60274,1
60275,1
60276,1
60277,1


In [28]:
X_scale = (X-X.mean())/X.std()

In [29]:
lr = LogisticRegression(fit_intercept=True)

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y, test_size=0.4,random_state=42) 

Randomly divide into train and test dataset

In [31]:
lr.fit(X_train,Y_train)

C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [32]:
preds=lr.predict(X_test)
preds

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [33]:
score = lr.score(X_test, Y_test)
score

0.5379479097544791

Elevation is not a good predictor at all.